#### Step 1: Load Ground Truth Dataset

In [4]:
import pandas as pd
df_ground_truth = pd.read_csv('ground-truth-data.csv')
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [6]:
ground_truth[10]

{'question': 'Are sessions recorded if I miss one?',
 'course': 'machine-learning-zoomcamp',
 'document': '5170565b'}

In [7]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [8]:
doc_idx = {d['id']: d for d in documents}
doc_idx['5170565b']['course']

'machine-learning-zoomcamp'

#### Step 2: Create Embeddings using Pretrained Models

Sentence Transformers documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [9]:

from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-MiniLM-L6-cos-v1'

model = SentenceTransformer(model_name)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
len(model.encode("This is a simple sentence"))

384

In [11]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'id': '1f6520ca'}

#### Step 3: Setup ElasticSearch connection 

In [12]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'b8d182300f91', 'cluster_name': 'docker-cluster', 'cluster_uuid': '_3--bU5ORBmSPSTkZjsNsw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

#### Step 4: Create Mappings and Index

* Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

* Each document is a collection of fields, which each have their own data type.

* We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored 



In [13]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_text_vector": {
                "type": "dense_vector", 
                            "dims": 384, 
                            "index": True,
                              "similarity": "cosine"},
        }
    }
}

In [14]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [15]:
from tqdm.auto import tqdm

In [16]:
for doc in tqdm(documents):
    # Transforming the title into an embedding using the model
    question = doc["question"]
    text = doc['text']
    doc["question_text_vector"] = model.encode(question+ ' ' + text)
    es_client.index(index= index_name, document= doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:59<00:00, 16.01it/s]


### Searching based on knn

In [17]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def question_text_vector_knn(q):
    question = q['question']
    course = q['course']
   
    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)



In [18]:
question_text_vector_knn(dict
                         (question = "What if I miss a session?", 
                          course= 'machine-learning-zoomcamp'))

[{'question': 'What if I miss a session?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.',
  'id': '5170565b'},
 {'question': 'Useful Resource for Missing Data Treatment\nhttps://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python/notebook',
  'course': 'machine-learning-zoomcamp',
  'section': '2. Machine Learning for Regression',
  'text': '(Hrithik Kumar Advani)',
  'id': '81b8e8d0'},
 {'question': 'Is it going to be live? When?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your qu

### Building RAG Flow 

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [21]:
from openai import OpenAI



client = OpenAI(api_key="enter your key")




In [22]:
def llm(prompt):
    response = client.chat.completions.create(
      model="gpt-4o-mini-2024-07-18",
      messages=[
        {
          "role": "user",
          "content": prompt,
        },
      ],
    )
    return response.choices[0].message.content

In [23]:
def rag(query: dict) -> str:
    search_results = question_text_vector_knn(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [24]:
rag(ground_truth[10])

"Yes, all sessions are recorded, so if you miss one, you won't miss anything important. You can also ask questions in advance for office hours, which will be covered during the live stream."

In [25]:
doc_idx['5170565b']['text']

'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.'

### Cosine Similarity

In [30]:
answers = {}
for i, rec in enumerate(tqdm(ground_truth)) :
    if i in answers: 
        continue
    answer_llm = rag(rec)
    doc_id = rec['document']
    answer_orig = doc_idx[doc_id]['text'] 
    answers[i]= {
        'answer_llm' : answer_llm,
        'answer_orignal': answer_orig,
        'document':  doc_id
        }


100%|██████████| 1830/1830 [37:03<00:00,  1.22s/it] 


In [39]:
result_gpt_4 = pd.DataFrame(answers.values())

In [41]:
result_gpt_4.to_csv("gtp4_dataframe", index = False)

In [35]:
result_gpt_4 = [None] * len(ground_truth)

In [36]:
for i, val in answers.items():
    result_gpt_4[i] = val.copy()
    result_gpt_4[i].update(ground_truth[i])

In [38]:
result_gpt_4[0]

{'answer_llm': 'You can sign up for the course "Machine Learning Zoomcamp" by visiting the course page at http://mlzoomcamp.com/.',
 'answer_orignal': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

### Evaluating GPT4-o-mino